#  GA-BPNN using LM optimization function or other

In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import BPNN
import GA


In [2]:
data = pd.read_csv('./Climb_train_data.csv')
test_data = pd.read_csv('./Climb_test_data.csv')

Climb_data = pd.concat([data, test_data],axis =0).reset_index(drop=True)

In [3]:
sc_x_train = MinMaxScaler(feature_range=(0, 1))
sc_y_train = MinMaxScaler(feature_range=(0, 1))
Climb_data_x = Climb_data.drop(labels=['FF','FF1C','FF2C'],axis=1).copy() 
sc_x_train.fit(Climb_data_x)
Climb_data_y = Climb_data[['FF']].copy()
sc_y_train.fit(Climb_data_y)

X = data.drop(labels=['FF','FF1C','FF2C'],axis=1).copy() 
X_stand = sc_x_train.transform(X)
x_train = torch.tensor(data=X_stand)

Y = data[['FF']].copy()
Y_stand = sc_y_train.transform(Y)
y_train = torch.tensor(data=Y_stand)

In [4]:
input_size, output_size = x_train.shape[1], y_train.shape[1]
hidden_size = 10
alpha = 0.01

In [5]:
model = BPNN.OneHiddenLayerNet(input_size,hidden_size,output_size)

In [6]:
def chrom_to_params(chrom):
    """ 染色体转码为模型参数 """
    params_template = copy.deepcopy(model.state_dict())
    params = copy.deepcopy(params_template)
    idx = 0
    for key in params:
        params_len = np.prod(params_template[key].shape)
        param = torch.from_numpy(chrom[idx: idx+params_len]).reshape(params_template[key].shape)
        params[key] = param
        idx += params_len
    return params

In [7]:
def calculate_fitness(chrom):
    """ GA适应性函数 """
    params = chrom_to_params(chrom)
    model.load_state_dict(params)
#     loss = BPNN.train_model_lrfix(x_train,y_train,model,learning_rate=0.1,m=0.9)[0]
    loss = BPNN.levenberg_marquardt(model, x_train, y_train,alpha)[0]
    print("loss:{} ".format(loss))
    fitness = 1./loss
    return fitness

In [36]:
pop_size = 100
num_len = 50
bound = [-1,+1]

# 编码长度
params_template = copy.deepcopy(model.state_dict())
chrom_len = 0
for key in params_template:
    params_len = np.prod(params_template[key].shape)
    chrom_len += params_len

In [8]:
ga = GA.Genetic_Algorithm(pop_size, chrom_len, bound, calculate_fitness, GenomeClass = GA.GenomeBinary, cross_prob=0.85, mutate_prob=0.015)

In [9]:
ga.genetic(num_len, log=True)

In [11]:
best_ga_params = chrom_to_params(ga.result())
torch.save(best_ga_params,'LM-0.01-10.pkl')

In [12]:
model_best = BPNN.OneHiddenLayerNet(input_size,hidden_size,output_size)
# model_best.load_state_dict(best_ga_params)
model_best.load_state_dict(torch.load('LM-0.01-10.pkl'))
# BPNN.train_model_lrfix(x_train,y_train,model_best,learning_rate=0.1,m=0.9)
BPNN.levenberg_marquardt(model_best, x_train, y_train, alpha)

(9.208651730323539e-05,
 [0.0007310089655235172,
  0.0005034781448731188,
  0.0002352954613583438,
  0.00020490749483693138,
  0.00019750648197453444,
  0.0001707963187034154,
  0.00016298176707968653,
  0.00015310003919326187,
  0.0001529620938139316,
  0.00011234537348629434,
  8.352054903561462e-05,
  7.937610350652702e-05,
  7.790599937030348e-05,
  7.606783693683181e-05,
  7.486008353427795e-05,
  7.391771428386759e-05,
  7.312070713606167e-05,
  7.146401995658079e-05,
  6.944389895232047e-05,
  5.870413387394314e-05,
  5.802218181173694e-05,
  5.611545586585596e-05,
  5.567402184925219e-05,
  5.4485424256879917e-05,
  5.4347153780398016e-05,
  5.343204399750851e-05,
  5.29121283769102e-05,
  4.7315408486810766e-05,
  4.666927763830099e-05,
  4.618989825550882e-05,
  4.600095419603136e-05,
  4.591035243174514e-05,
  4.533568915809548e-05,
  4.31386642514161e-05,
  4.008871893855752e-05,
  3.8934722831520886e-05,
  3.798996023889402e-05,
  3.7823435780678714e-05,
  3.75925096876470

In [13]:
y_pred = model_best(x_train)
Loss = nn.MSELoss(reduction='mean')
loss = Loss(y_pred, y_train)
print(loss.item())

3.5319151786473905e-05
